<a href="https://colab.research.google.com/github/YoungsikMoon/FORS/blob/main/%EC%A1%B0%EC%98%81%EC%88%98/FORS_%EC%A1%B0%EC%98%81%EC%88%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Whisper AI fine-tuning 하는 코드

WhisperAI_fine_tune_version6

## LoRA (Low Rank Adaptation) 와 PEFT (Paramater Efficient Fine Tuning)를 적용한 Larger Whisper fine-tuning ⚡️



* 소비자 GPU의 VRAM이 8GB 미만인 환경에서도 full-finetuning과 유사한 성능을 제공함
*  🤗 Transformers and PEFT 모델과 Common Voice 13.0 dataset를 사용하여 Whisper fine-tune하는 과정 설명함
* PEFT와 bitsandbytes를 활용하여 무료 T4 GPU(16GB VRAM)를 사용하여 whisper-large-v2 체크포인트를 원활하게 학습

### 왜 Parameter Efficient Fine Tuning [PEFT](https://github.com/huggingface/peft)를 사용해야 되는가?


* 모델 사이즈 증가로 fine tuning하는 것이 계산 복잡성 증가와 메모리 사용량 증가
    * 예를 들어, Whisper-large-v2 모델을 완전한 미세 조정을 위해 약 24GB의 GPU VRAM이 필요하며, 각 미세 조정된 모델은 약 7GB의 저장 공간을 필요함

    * 제한적인 환경에서 bottleneck 발생하고 원하는 결과를 얻기 힘듦
* PEFT
    * 효과적으로 parameter를 줄여서 fine tuning 속도 개선
    * 목적: 병목 현상을 해결
    * 접근법(예: 저수준 적응): 사전 훈련된 모델의 대부분의 매개변수를 동결시키면서 추가 모델 매개변수의 일부만 미세 조정하여 계산 및 저장 비용 크게 줄임
        * 대규모 모델의 전체 미세 조정 중 관찰되는 catastrophic forgetting 문제를 극복할 수 있음



    





#### LoRA가 무엇인가?



* PEFT에서 여러 매개변수 효율적인 기술을 기본으로 제공
    * 그 중 하나인 Low Rank Adaptation (LoRA)
        * 사전 훈련된 모델 가중치를 동결하고 Transformer 아키텍처의 각 레이어에 훈련 가능한 랭크 분해 행렬을 삽입 (High Rank 즉 많은 연결이 되어 있는 것들보다 연결이 적은 Low Rank로 만들어서 계산량을 줄임)
            * Downstream 작업에 대한 훈련 가능한 매개변수 수가 크게 감소


#### 통계로 보는 PEFT 효과



* Full fine-tuning of Whisper-large-v2 checkpoint Vs. PEFT 적용 모델

    1. GPU VRAM이 8GB 미만인 환경에서 16억 개의 매개변수를 가진 모델을 미세 조정 🤯
    2. 훨씬 적은 수의 훈련 가능한 매개변수를 사용하여 거의 5배 더 큰 배치 크기를 사용 가능 📈
    3. 생성된 체크포인트는 원본 모델의 크기의 1%인 약 60MB 🚀
* 기존 🤗 transformers Whisper에서 변형이 많이 되지 않았음

### 환경설정




* Python package->Whisper 모델 fine tuning하기 위해 사용
  * `datasets`:학습 데이터를 다운로드하고 준비
  * `transformers`: Whisper 모델을 로드하고 훈련
  * `librosa`: 오디오 파일을 전처리
  * `evaluate` &  `jiwer`:모델의 성능을 평가
  * `PEFT`, `bitsandbytes`, `accelerate`: LoRA로 모델과 fine-tuning

In [ ]:
!pip install -q transformers datasets librosa evaluate jiwer gradio bitsandbytes==0.37 accelerate
!pip install -q git+https://github.com/huggingface/peft.git@main

In [ ]:
!pip install typer==0.9.1


* GPU 확보
    * Google Colab Pro: V100 또는 P100 GPU가 할당
* GPU 확보 방법
    * 런타임 -> 런타임 유형 변경
    * None -> GPU 변경
* GPU 할당 확인

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Colab에서 제공하는 GPU사용

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

We strongly advise you to upload model checkpoints directly the [Hugging Face Hub](https://huggingface.co/)
whilst training. The Hub provides:
- Integrated version control: you can be sure that no model checkpoint is lost during training.
- Tensorboard logs: track important metrics over the course of training.
- Model cards: document what a model does and its intended use cases.
- Community: an easy way to share and collaborate with the community!


* 학습 중 모델 체크포인트를 직접 Hugging Face Hub에 업로드하는 것을 강력히 권장
 * Hub를 사용하면 다음과 같은 기능을 제공:

 1. 통합된 버전 관리: 훈련 중에 어떠한 모델 체크포인트도 손실되지 않음
 2. Tensorboard 로그: 훈련 과정에서 중요한 지표를 추적
 3. 모델 카드: 모델이 무엇을 하고 의도된 사용 사례를 문서화
 4. 커뮤니티: 커뮤니티와 쉽게 공유하고 협업

* Hub에 연결
 * 프롬프트에서 Hub 인증 토큰을 입력:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


Whisper 모델 checkpoint와 task 설정

In [ ]:
model_name_or_path = "openai/whisper-large-v2"
task = "transcribe"

데이터셋 디테일을 설정 (언어)

In [ ]:
dataset_name = "mozilla-foundation/common_voice_13_0"
language = "Korean"
language_abbr = "ko" # Short hand code for the language we want to fine-tune

## 데이터셋 올리기



* Huggingface Dataset 사용
 * 적은 코드로 Common Voice의 데이터를 다운로드하고 준비

* 확인 절차
 1. Hugging Face Hub의 이용 약관을 수락했는지 확인:[mozilla-foundation/common_voice_13_0](https://huggingface.co/datasets/mozilla-foundation/common_voice_13_0)
 2. 데이터셋에 액세스하고 로컬로 데이터를 다운로드

* 학습+검증 데이터셋/테스트 데이터셋 분리

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset(dataset_name, language_abbr, split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset(dataset_name, language_abbr, split="test", use_auth_token=True)

print(common_voice)

* 일반적인 ASR(음성 인식) 데이터셋
    * 입력 오디오 샘플(오디오)과 해당되는 텍스트(문장)만 제공
* Common Voice
    * ASR에는 필요하지 않은 악센트와 로케일과 같은 추가 메타데이터 정보가 포함
    * 일반적인 용도로 사용하고 미세 조정을 고려하기 위해 메타데이터 정보 무시

In [ ]:
common_voice = common_voice.remove_columns(
    ["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes", "variant"]
)

print(common_voice)

### 특성 추출기(Feature Extractor), 토크나이저(Tokenizer), 그리고 데이터준비



ASR 파이프라인 세 단계로 분해:

1. Raw 오디오 입력을 전처리하는 특정 추출기
2. 시퀀스 간 매핑을 수행하는 모델
3. 모델 출력을 텍스트 형식으로 후처리하는 tokenizer


* Whisper
    * [WhisperFeatureExtractor](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperFeatureExtractor)와 [WhisperTokenizer](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperTokenizer)로 구성



In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=language, task=task)

* WhisperProcessor 클라스
    * 특성 추출기와 토크나이저를 사용하기 위해 두 가지를 모두 합칩
    * 필요에 따라 오디오 입력 및 모델 예측에 사용 가능

* 학습 중에 두 개의 객체만 추적 필요: 프로세서와 모델

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)

#### 데이터 준비



Common Voice 데이터셋의 첫 번째 예제를 출력하여 데이터의 형식을 살펴봄

In [ ]:
print(common_voice["train"][0])

* Whisper 모델 샘플링
    * 입력 오디오는 48 kHz 새플링
    * Whisper feature extractor에 전달하기 위해서 16 kHz로 다운샘플 진행
* 샘플링 속도 설정
    * Dataset의 [`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column) 방법 사용: 오디오 입력을 올바른 샘플링 속도로 설정
    * 오디오를 변경하는 것이 아니라 오디오 샘플을 실시간으로 받을 수 있도록 함


In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))


Common Voice 데이터셋에서 첫 번째 오디오 샘플을 다시로드하면 원하는 샘플링 속도로 다시 샘플링

In [ ]:
print(common_voice["train"][0])


* 모델에 맞게 데이터를 준비하는 함수:

1. batch["audio"]를 호출하여 오디오 데이터를 로드하고 다시 샘플링. 🤗 Datasets는 필요한 모든 재샘플링 작업을 실시간으로 수행
3. Feature extractor를 사용하여 1차원 오디오 배열에서 로그 멜 스펙트로그램 입력 특성을 계산
3. Tokenizer를 사용하여 transcripts를 레이블 ids로 인코딩


In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

* 데이터 준비 함수
    * 데이터셋의 .map 메서드를 사용하여 모든 학습 예제에 적용 가능
    * num_proc: 몇 개의 CPU 코어를 사용할 지를 지정,num_proc를 1보다 크게 설정하면 다중 처리가 활성화 (다중 처리로 .map 메서드가 중단되는 경우 num_proc=1로 설정하고 데이터셋을 순차적으로 처리)

* Dataset의 사이즈에 따라서 20~30 분 정도 걸림


In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

In [ ]:
common_voice["train"]

### 학습 및 검증





* 훈련 파이프라인
* [🤗 Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer)가 대부분의 작업을 처리:


1. 데이터 collator 정의: 데이터 콜레이터는 우리가 전처리한 데이터를 가져와 모델에 사용할 수 있는 PyTorch 텐서로 준비
2. 평가 지표: 평가 중에는 모델을 글자 오류율  [word error rate (CER)](https://huggingface.co/metrics/cer)지표를 사용하여 평가
3. 사전 훈련된 체크포인트 load: 사전 훈련된 체크포인트를 로드하고 훈련을 위해 올바르게 구성
4. 훈련 구성 정의: 🤗 Trainer가 훈련 스케줄을 정의에 사용

* 미세 조정한 후에는 테스트 데이터에서 모델을 평가하여 한국어 음성을 올바르게 transcribe

### Data Collator 정의



* 시퀀스-투-시퀀스 음성 모델의 데이터 콜레이터
    * Input_features와 labels를 독립적으로 처리 차별화
    
    - Input_features: feature extractor에 의해 처리
    - labels: tokenizer에 의해 처리

* Input_features는 이미 30초로 패딩되어 있고 특성 추출기에 의해 고정된 차원의 로그 멜 스펙트로그램으로 변환. 따라서 우리가 해야 할 일은 input_features를 배치 처리된 PyTorch 텐서로 변환

* labels는 패딩되지 않음 먼저 배치 내에서 최대 길이에 맞게 시퀀스를 패딩하고, tokenizer의 .pad 메서드를 사용하여 시퀀스를 패딩 패딩 토큰은 손실을 계산할 때 고려되지 않도록 -100으로 대체. 그런 다음 레이블 시퀀스의 시작에서 BOS 토큰을 잘라서 훈련 중에 나중에 이를 추가

* 이전에 정의한 WhisperProcessor를 활용하여 특성 추출기 및 토크나이저 작업을 모두 수행 가능

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Data collator 초기화 진행

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

#### 평가 지표


* ASR 시스템을 평가하기 위한 '사실상의' 지표인 한 단어 오류율(CER) 메트릭을 사용
* 더 많은 정보는 [문서](https://huggingface.co/metrics/cer)를 참조. 우리는 🤗 Evaluate에서 CER 메트릭을 로드

In [ ]:
import evaluate

metric = evaluate.load("cer")

#### Pre-trained 모델 로드

* 사전 훈련된 Whisper 체크포인트를 로드
    * 이 작업은 🤗 Transformers를 사용하여 매우 간단



* 모델의 메모리 사용량을 줄이기 위해 모델을 8비트로         
    * 모델을 1/4 정밀도(32비트와 비교했을 때)로 양자화하여 성능 손실을 최소화 [here](https://huggingface.co/blog/hf-bitsandbytes-integration)

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path, load_in_8bit=True, device_map="auto")

#### 모델의 후처리



1. 훈련을 가능하게 하기 위해 8비트 모델에 몇 가지 후처리 단계를 적용
2. 모델 레이어를 동결, 훈련과 모델의 안정성을 위해 레이어 정규화와 출력 레이어를 float32로 캐스팅

(모델 안정성과 layer normalization 분석, float32로 캐스팅 하는 이유)

In [ ]:
!pip install peft

In [ ]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model, output_embedding_layer_name="proj_out")

* Whisper 모델은 인코더에 컨볼루션 레이어를 사용하기 때문에 체크포인팅은 grad 연산을 비활성. 이를 피하기 위해 입력을 특별히 trainable하게 만들어야 합니다.


In [ ]:
def make_inputs_require_grad(module, input, output):
    output.requires_grad_(True)

model.model.encoder.conv1.register_forward_hook(make_inputs_require_grad)

#### Low-rank adapters (LoRA)를 모델에 적용



* peft에서 get_peft_model 유틸리티 함수를 사용하여 PeftModel을 로드하고 저희가 저차원 어댑터(LoRA)를 사용할 것임을 지정


In [ ]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

**1%**의 학습 parameter를 사용하였고 **Parameter-Efficient Fine-Tuning**를 적용


#### 훈련 구성 정의


마지막 단계에서는 훈련과 관련된 모든 매개변수를 정의 훈련 인자에 대한 자세한 내용은 해당 문서를 참조 Seq2SeqTrainingArguments [docs](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments)


In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="reach-vb/test",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-3,
    warmup_steps=50,
    num_train_epochs=1,
    evaluation_strategy="steps",
    fp16=True,
    per_device_eval_batch_size=8,
    generation_max_length=128,
    logging_steps=100,
    max_steps=100, # only for testing purposes, remove this from your final run :)
    remove_unused_columns=False,  # required as the PeftModel forward doesn't have the signature of the wrapped model's forward
    label_names=["labels"],  # same reason as above
)


* PEFT를 사용하여 모델을 미세 조정하는 것에는 몇 가지 주의가 필요

1. PeftModel의 forward가 기본 모델의 forward의 시그니처를 상속하지 않기 때문에 remove_unused_columns=False 및 label_names=["labels"]를 명시적으로 설정
2. INT8 훈련에는 자동 캐스팅이 필요하기 때문에 Trainer에서 기본적으로 제공되는 predict_with_generate 호출을 사용할 수 없습니다. 자동 캐스팅이 자동으로 적용되지 않음
3. 자동 캐스팅을 사용할 수 없으므로 Seq2SeqTrainer에 compute_metrics를 전달할 수 없음. 따라서 Trainer를 인스턴스화하는 동안 compute_metrics를 주석 처리해야 합니다.



In [ ]:
from transformers import Seq2SeqTrainer, TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR

# This callback helps to save only the adapter weights and remove the base model weights.
class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control


trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[SavePeftModelCallback],
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [ ]:
trainer.train()

Now that our model is fine-tuned, we can push

*   항목 추가
*   항목 추가

the model on to Hugging Face Hub, this will later help us directly infer the model from the model repo.

## Evaluation and Inference

On to the fun part, we've successfully fine-tuned our model. Now let's put it to test and calculate the WER on the `test` set.

As with training, we do have a few caveats to pay attention to:
1. Since we cannot use `predict_with_generate` function, we will hand roll our own eval loop with `torch.cuda.amp.autocast()` you can check it out below.
2. Since the base model is frozen, PEFT model sometimes fails to recognise the language while decoding. To fix that, we force the starting tokens to mention the language we are transcribing. This is done via `forced_decoder_ids = processor.get_decoder_prompt_ids(language="Marathi", task="transcribe")` and passing that too the `model.generate` call.

That's it, let's get transcribing! 🔥


In [ ]:
from peft import PeftModel, PeftConfig
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer

peft_model_id = "reach-vb/whisper-large-v2-hindi-100steps" # Use the same model ID as before.
peft_config = PeftConfig.from_pretrained(peft_model_id)
model = WhisperForConditionalGeneration.from_pretrained(
    peft_config.base_model_name_or_path, load_in_8bit=True, device_map="auto"
)
model = PeftModel.from_pretrained(model, peft_model_id)
model.config.use_cache = True

In [ ]:
import gc
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

eval_dataloader = DataLoader(common_voice["test"], batch_size=8, collate_fn=data_collator)
forced_decoder_ids = processor.get_decoder_prompt_ids(language=language, task=task)
normalizer = BasicTextNormalizer()

predictions = []
references = []
normalized_predictions = []
normalized_references = []

model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            generated_tokens = (
                model.generate(
                    input_features=batch["input_features"].to("cuda"),
                    forced_decoder_ids=forced_decoder_ids,
                    max_new_tokens=255,
                )
                .cpu()
                .numpy()
            )
            labels = batch["labels"].cpu().numpy()
            labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)
            decoded_preds = processor.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            decoded_labels = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)
            predictions.extend(decoded_preds)
            references.extend(decoded_labels)
            normalized_predictions.extend([normalizer(pred).strip() for pred in decoded_preds])
            normalized_references.extend([normalizer(label).strip() for label in decoded_labels])
        del generated_tokens, labels, batch
    gc.collect()
wer = 100 * metric.compute(predictions=predictions, references=references)
normalized_wer = 100 * metric.compute(predictions=normalized_predictions, references=normalized_references)
eval_metrics = {"eval/wer": wer, "eval/normalized_wer": normalized_wer}

print(f"{wer=} and {normalized_wer=}")
print(eval_metrics)

### Fin!

If you made it all the way till the end then pat yourself on the back. Looking back, we learned how to train *any* Whisper checkpoint faster, cheaper and with negligible loss in WER.

With PEFT, you can also go beyond Speech recognition and apply the same set of techniques to other pretrained models as well. Come check it out here: https://github.com/huggingface/peft 🤗

Don't forget to tweet your results and tag us! [@huggingface](https://twitter.com/huggingface) and [@reach_vb](https://twitter.com/reach_vb) ❤️

#  Whisper AI fine-tuning의 경량화 버전

fasterWhisper_w_PEFT_finetune (quantization과 parameter efficient fine-tuning 기법)

## LoRA (Low Rank Adaptation) 와 PEFT (Paramater Efficient Fine Tuning)를 적용한 Larger Whisper fine-tuning ⚡️


* 소비자 GPU의 VRAM이 8GB 미만인 환경에서도 full-finetuning과 유사한 성능을 제공함
*  🤗 Transformers and PEFT 모델과 Common Voice 13.0 dataset를 사용하여 Whisper fine-tune하는 과정 설명함
* PEFT와 bitsandbytes를 활용하여 무료 T4 GPU(16GB VRAM)를 사용하여 whisper-large-v2 체크포인트를 원활하게 학습

### 왜 Parameter Efficient Fine Tuning [PEFT](https://github.com/huggingface/peft)를 사용해야 되는가?


* 모델 사이즈 증가로 fine tuning하는 것이 계산 복잡성 증가와 메모리 사용량 증가
    * 예를 들어, Whisper-large-v2 모델을 완전한 미세 조정을 위해 약 24GB의 GPU VRAM이 필요하며, 각 미세 조정된 모델은 약 7GB의 저장 공간을 필요함

    * 제한적인 환경에서 bottleneck 발생하고 원하는 결과를 얻기 힘듦
* PEFT
    * 효과적으로 parameter를 줄여서 fine tuning 속도 개선
    * 목적: 병목 현상을 해결
    * 접근법(예: 저수준 적응): 사전 훈련된 모델의 대부분의 매개변수를 동결시키면서 추가 모델 매개변수의 일부만 미세 조정하여 계산 및 저장 비용 크게 줄임
        * 대규모 모델의 전체 미세 조정 중 관찰되는 catastrophic forgetting 문제를 극복할 수 있음


#### LoRA가 무엇인가?

* PEFT에서 여러 매개변수 효율적인 기술을 기본으로 제공
    * 그 중 하나인 Low Rank Adaptation (LoRA)
        * 사전 훈련된 모델 가중치를 동결하고 Transformer 아키텍처의 각 레이어에 훈련 가능한 랭크 분해 행렬을 삽입 (High Rank 즉 많은 연결이 되어 있는 것들보다 연결이 적은 Low Rank로 만들어서 계산량을 줄임)
            * Downstream 작업에 대한 훈련 가능한 매개변수 수가 크게 감소

#### 통계로 보는 PEFT 효과


* Full fine-tuning of Whisper-large-v2 checkpoint Vs. PEFT 적용 모델

    1. GPU VRAM이 8GB 미만인 환경에서 16억 개의 매개변수를 가진 모델을 미세 조정 🤯
    2. 훨씬 적은 수의 훈련 가능한 매개변수를 사용하여 거의 5배 더 큰 배치 크기를 사용 가능 📈
    3. 생성된 체크포인트는 원본 모델의 크기의 1%인 약 60MB 🚀
* 기존 🤗 transformers Whisper에서 변형이 많이 되지 않았음

    



### 환경설정


* Python package->Whisper 모델 fine tuning하기 위해 사용
  * `datasets`:학습 데이터를 다운로드하고 준비
  * `transformers`: Whisper 모델을 로드하고 훈련
  * `librosa`: 오디오 파일을 전처리
  * `evaluate` &  `jiwer`:모델의 성능을 평가
  * `PEFT`, `bitsandbytes`, `accelerate`: LoRA로 모델과 fine-tuning

In [ ]:
!pip install -q transformers datasets librosa evaluate jiwer gradio bitsandbytes==0.37 accelerate
!pip install -q git+https://github.com/huggingface/peft.git@main

In [ ]:
!pip install typer==0.9.1


* GPU 확보
    * Google Colab Pro: V100 또는 P100 GPU가 할당
* GPU 확보 방법
    * 런타임 -> 런타임 유형 변경
    * None -> GPU 변경
* GPU 할당 확인

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Colab에서 제공하는 GPU사용

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

We strongly advise you to upload model checkpoints directly the [Hugging Face Hub](https://huggingface.co/)
whilst training. The Hub provides:
- Integrated version control: you can be sure that no model checkpoint is lost during training.
- Tensorboard logs: track important metrics over the course of training.
- Model cards: document what a model does and its intended use cases.
- Community: an easy way to share and collaborate with the community!


* 학습 중 모델 체크포인트를 직접 Hugging Face Hub에 업로드하는 것을 강력히 권장
 * Hub를 사용하면 다음과 같은 기능을 제공:

 1. 통합된 버전 관리: 훈련 중에 어떠한 모델 체크포인트도 손실되지 않음
 2. Tensorboard 로그: 훈련 과정에서 중요한 지표를 추적
 3. 모델 카드: 모델이 무엇을 하고 의도된 사용 사례를 문서화
 4. 커뮤니티: 커뮤니티와 쉽게 공유하고 협업

* Hub에 연결
 * 프롬프트에서 Hub 인증 토큰을 입력:

from huggingface_hub import notebook_login

notebook_login()


Whisper 모델 checkpoint와 task 설정

In [ ]:
model_name_or_path = "openai/whisper-large-v2"
task = "transcribe"



데이터셋 디테일을 설정 (언어)

In [ ]:
dataset_name = "mozilla-foundation/common_voice_13_0"
language = "Korean"
language_abbr = "ko" # Short hand code for the language we want to fine-tune

## 데이터셋 올리기



* Huggingface Dataset 사용
 * 적은 코드로 Common Voice의 데이터를 다운로드하고 준비

* 확인 절차
 1. Hugging Face Hub의 이용 약관을 수락했는지 확인:[mozilla-foundation/common_voice_13_0](https://huggingface.co/datasets/mozilla-foundation/common_voice_13_0)
 2. 데이터셋에 액세스하고 로컬로 데이터를 다운로드

* 학습+검증 데이터셋/테스트 데이터셋 분리

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset(dataset_name, language_abbr, split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset(dataset_name, language_abbr, split="test", use_auth_token=True)

print(common_voice)

* 일반적인 ASR(음성 인식) 데이터셋
    * 입력 오디오 샘플(오디오)과 해당되는 텍스트(문장)만 제공
* Common Voice
    * ASR에는 필요하지 않은 악센트와 로케일과 같은 추가 메타데이터 정보가 포함
    * 일반적인 용도로 사용하고 미세 조정을 고려하기 위해 메타데이터 정보 무시

In [ ]:
common_voice = common_voice.remove_columns(
    ["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes", "variant"]
)

print(common_voice)

### 특성 추출기(Feature Extractor), 토크나이저(Tokenizer), 그리고 데이터준비



ASR 파이프라인 세 단계로 분해:

1. Raw 오디오 입력을 전처리하는 특정 추출기
2. 시퀀스 간 매핑을 수행하는 모델
3. 모델 출력을 텍스트 형식으로 후처리하는 tokenizer


* Whisper
    * [WhisperFeatureExtractor](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperFeatureExtractor)와 [WhisperTokenizer](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperTokenizer)로 구성



In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name_or_path)

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained(model_name_or_path, language=language, task=task)

* WhisperProcessor 클라스
    * 특성 추출기와 토크나이저를 사용하기 위해 두 가지를 모두 합칩
    * 필요에 따라 오디오 입력 및 모델 예측에 사용 가능

* 학습 중에 두 개의 객체만 추적 필요: 프로세서와 모델

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained(model_name_or_path, language=language, task=task)

#### 데이터 준비



Common Voice 데이터셋의 첫 번째 예제를 출력하여 데이터의 형식을 살펴봄

In [ ]:
print(common_voice["train"][0])

* Whisper 모델 샘플링
    * 입력 오디오는 48 kHz 새플링
    * Whisper feature extractor에 전달하기 위해서 16 kHz로 다운샘플 진행
* 샘플링 속도 설정
    * Dataset의 [`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column) 방법 사용: 오디오 입력을 올바른 샘플링 속도로 설정
    * 오디오를 변경하는 것이 아니라 오디오 샘플을 실시간으로 받을 수 있도록 함


In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))


Common Voice 데이터셋에서 첫 번째 오디오 샘플을 다시로드하면 원하는 샘플링 속도로 다시 샘플링

In [ ]:
print(common_voice["train"][0])


* 모델에 맞게 데이터를 준비하는 함수:

1. batch["audio"]를 호출하여 오디오 데이터를 로드하고 다시 샘플링. 🤗 Datasets는 필요한 모든 재샘플링 작업을 실시간으로 수행
3. Feature extractor를 사용하여 1차원 오디오 배열에서 로그 멜 스펙트로그램 입력 특성을 계산
3. Tokenizer를 사용하여 transcripts를 레이블 ids로 인코딩


In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

* 데이터 준비 함수
    * 데이터셋의 .map 메서드를 사용하여 모든 학습 예제에 적용 가능
    * num_proc: 몇 개의 CPU 코어를 사용할 지를 지정,num_proc를 1보다 크게 설정하면 다중 처리가 활성화 (다중 처리로 .map 메서드가 중단되는 경우 num_proc=1로 설정하고 데이터셋을 순차적으로 처리)

* Dataset의 사이즈에 따라서 20~30 분 정도 걸림


In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

In [ ]:
common_voice["train"]

### 학습 및 검증




* 훈련 파이프라인
* [🤗 Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer)가 대부분의 작업을 처리:


1. 데이터 collator 정의: 데이터 콜레이터는 우리가 전처리한 데이터를 가져와 모델에 사용할 수 있는 PyTorch 텐서로 준비
2. 평가 지표: 평가 중에는 모델을 글자 오류율  [word error rate (CER)](https://huggingface.co/metrics/cer)지표를 사용하여 평가
3. 사전 훈련된 체크포인트 load: 사전 훈련된 체크포인트를 로드하고 훈련을 위해 올바르게 구성
4. 훈련 구성 정의: 🤗 Trainer가 훈련 스케줄을 정의에 사용

* 미세 조정한 후에는 테스트 데이터에서 모델을 평가하여 한국어 음성을 올바르게 transcribe


#### Data Collator 정의



* 시퀀스-투-시퀀스 음성 모델의 데이터 콜레이터
    * Input_features와 labels를 독립적으로 처리 차별화
    
    - Input_features: feature extractor에 의해 처리
    - labels: tokenizer에 의해 처리

* Input_features는 이미 30초로 패딩되어 있고 특성 추출기에 의해 고정된 차원의 로그 멜 스펙트로그램으로 변환. 따라서 우리가 해야 할 일은 input_features를 배치 처리된 PyTorch 텐서로 변환

* labels는 패딩되지 않음 먼저 배치 내에서 최대 길이에 맞게 시퀀스를 패딩하고, tokenizer의 .pad 메서드를 사용하여 시퀀스를 패딩 패딩 토큰은 손실을 계산할 때 고려되지 않도록 -100으로 대체. 그런 다음 레이블 시퀀스의 시작에서 BOS 토큰을 잘라서 훈련 중에 나중에 이를 추가

* 이전에 정의한 WhisperProcessor를 활용하여 특성 추출기 및 토크나이저 작업을 모두 수행 가능

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Data collator 초기화 진행

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

#### 평가 지표


* ASR 시스템을 평가하기 위한 '사실상의' 지표인 한 단어 오류율(CER) 메트릭을 사용
* 더 많은 정보는 [문서](https://huggingface.co/metrics/cer)를 참조. 우리는 🤗 Evaluate에서 CER 메트릭을 로드

In [ ]:
import evaluate

metric = evaluate.load("cer")

#### Pre-trained 모델 로드

* 사전 훈련된 Whisper 체크포인트를 로드
    * 이 작업은 🤗 Transformers를 사용하여 매우 간단



* 모델의 메모리 사용량을 줄이기 위해 모델을 8비트로         
    * 모델을 1/4 정밀도(32비트와 비교했을 때)로 양자화하여 성능 손실을 최소화 [here](https://huggingface.co/blog/hf-bitsandbytes-integration)

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(model_name_or_path, load_in_8bit=True, device_map="auto")

#### 모델의 후처리


1. 훈련을 가능하게 하기 위해 8비트 모델에 몇 가지 후처리 단계를 적용
2. 모델 레이어를 동결, 훈련과 모델의 안정성을 위해 레이어 정규화와 출력 레이어를 float32로 캐스팅

(모델 안정성과 layer normalization 분석, float32로 캐스팅 하는 이유)

In [ ]:
!pip install peft

In [ ]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model, output_embedding_layer_name="proj_out")

* Whisper 모델은 인코더에 컨볼루션 레이어를 사용하기 때문에 체크포인팅은 grad 연산을 비활성. 이를 피하기 위해 입력을 특별히 trainable하게 만들어야 합니다.


In [ ]:
def make_inputs_require_grad(module, input, output):
    output.requires_grad_(True)

model.model.encoder.conv1.register_forward_hook(make_inputs_require_grad)

#### Low-rank adapters (LoRA)를 모델에 적용



* peft에서 get_peft_model 유틸리티 함수를 사용하여 PeftModel을 로드하고 저희가 저차원 어댑터(LoRA)를 사용할 것임을 지정


In [ ]:
from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

config = LoraConfig(r=32, lora_alpha=64, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, config)
model.print_trainable_parameters()

**1%**의 학습 parameter를 사용하였고 **Parameter-Efficient Fine-Tuning**를 적용


#### 훈련 구성 정의


마지막 단계에서는 훈련과 관련된 모든 매개변수를 정의 훈련 인자에 대한 자세한 내용은 해당 문서를 참조 Seq2SeqTrainingArguments [docs](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments)


In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="reach-vb/test",  # change to a repo name of your choice
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-3,
    warmup_steps=50,
    num_train_epochs=1,
    evaluation_strategy="steps",
    fp16=True,
    per_device_eval_batch_size=8,
    generation_max_length=128,
    logging_steps=100,
    max_steps=100, # only for testing purposes, remove this from your final run :)
    remove_unused_columns=False,  # required as the PeftModel forward doesn't have the signature of the wrapped model's forward
    label_names=["labels"],  # same reason as above
)


* PEFT를 사용하여 모델을 미세 조정하는 것에는 몇 가지 주의가 필요

1. PeftModel의 forward가 기본 모델의 forward의 시그니처를 상속하지 않기 때문에 remove_unused_columns=False 및 label_names=["labels"]를 명시적으로 설정
2. INT8 훈련에는 자동 캐스팅이 필요하기 때문에 Trainer에서 기본적으로 제공되는 predict_with_generate 호출을 사용할 수 없습니다. 자동 캐스팅이 자동으로 적용되지 않음
3. 자동 캐스팅을 사용할 수 없으므로 Seq2SeqTrainer에 compute_metrics를 전달할 수 없음. 따라서 Trainer를 인스턴스화하는 동안 compute_metrics를 주석 처리해야 합니다.



In [ ]:
from transformers import Seq2SeqTrainer, TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR

# This callback helps to save only the adapter weights and remove the base model weights.
class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control


trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[SavePeftModelCallback],
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [ ]:
trainer.train()

Now that our model is fine-tuned, we can push the model on to Hugging Face Hub, this will later help us directly infer the model from the model repo.

In [ ]:
peft_model_id = "reach-vb/whisper-large-v2-hindi-100steps"
model.push_to_hub(peft_model_id)

## Evaluation and Inference

On to the fun part, we've successfully fine-tuned our model. Now let's put it to test and calculate the WER on the `test` set.

As with training, we do have a few caveats to pay attention to:
1. Since we cannot use `predict_with_generate` function, we will hand roll our own eval loop with `torch.cuda.amp.autocast()` you can check it out below.
2. Since the base model is frozen, PEFT model sometimes fails to recognise the language while decoding. To fix that, we force the starting tokens to mention the language we are transcribing. This is done via `forced_decoder_ids = processor.get_decoder_prompt_ids(language="Marathi", task="transcribe")` and passing that too the `model.generate` call.

That's it, let's get transcribing! 🔥


In [ ]:
from peft import PeftModel, PeftConfig
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer

peft_model_id = "reach-vb/whisper-large-v2-hindi-100steps" # Use the same model ID as before.
peft_config = PeftConfig.from_pretrained(peft_model_id)
model = WhisperForConditionalGeneration.from_pretrained(
    peft_config.base_model_name_or_path, load_in_8bit=True, device_map="auto"
)
model = PeftModel.from_pretrained(model, peft_model_id)
model.config.use_cache = True

In [ ]:
import gc
import numpy as np
from tqdm import tqdm
from torch.utils.data import DataLoader
from transformers.models.whisper.english_normalizer import BasicTextNormalizer

eval_dataloader = DataLoader(common_voice["test"], batch_size=8, collate_fn=data_collator)
forced_decoder_ids = processor.get_decoder_prompt_ids(language=language, task=task)
normalizer = BasicTextNormalizer()

predictions = []
references = []
normalized_predictions = []
normalized_references = []

model.eval()
for step, batch in enumerate(tqdm(eval_dataloader)):
    with torch.cuda.amp.autocast():
        with torch.no_grad():
            generated_tokens = (
                model.generate(
                    input_features=batch["input_features"].to("cuda"),
                    forced_decoder_ids=forced_decoder_ids,
                    max_new_tokens=255,
                )
                .cpu()
                .numpy()
            )
            labels = batch["labels"].cpu().numpy()
            labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)
            decoded_preds = processor.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            decoded_labels = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)
            predictions.extend(decoded_preds)
            references.extend(decoded_labels)
            normalized_predictions.extend([normalizer(pred).strip() for pred in decoded_preds])
            normalized_references.extend([normalizer(label).strip() for label in decoded_labels])
        del generated_tokens, labels, batch
    gc.collect()
wer = 100 * metric.compute(predictions=predictions, references=references)
normalized_wer = 100 * metric.compute(predictions=normalized_predictions, references=normalized_references)
eval_metrics = {"eval/wer": wer, "eval/normalized_wer": normalized_wer}

print(f"{wer=} and {normalized_wer=}")
print(eval_metrics)

### Fin!



If you made it all the way till the end then pat yourself on the back. Looking back, we learned how to train *any* Whisper checkpoint faster, cheaper and with negligible loss in WER.

With PEFT, you can also go beyond Speech recognition and apply the same set of techniques to other pretrained models as well. Come check it out here: https://github.com/huggingface/peft 🤗

Don't forget to tweet your results and tag us! [@huggingface](https://twitter.com/huggingface) and [@reach_vb](https://twitter.com/reach_vb) ❤️